In [1]:
import os, time
import math
import numpy as np
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import imageio

In [2]:
epochs = 5
batch_size = 64
latent_dim = 100
img_size = 64
IMAGE_PATH = "./celeba/img"

In [3]:
cuda = True if torch.cuda.is_available() else False

In [4]:
if not os.path.isdir('sample_images'):
    os.mkdir('sample_images')

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
class Generator(nn.Module):
    # initializers
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 16
        self.rel = nn.LeakyReLU(0.2, inplace=True)
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))
        self.deconv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.deconv1_bn = nn.BatchNorm2d(64)
        self.deconv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(32)
        self.deconv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(16)
        self.deconv4 = nn.ConvTranspose2d(16, 3, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = self.l1(input)
        x = x.view(x.shape[0], 128, self.init_size, self.init_size)
        x = self.rel(self.deconv1_bn(self.deconv1(x)))
        x = self.rel(self.deconv2_bn(self.deconv2(x)))
        x = self.rel(self.deconv3_bn(self.deconv3(x)))
        x = F.tanh(self.deconv4(x))

        return x

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.convs = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        ds_size = img_size // 2 ** 4
        self.fc_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
#         print(img.shape)
        x = self.convs(img)
#         print(x.shape)
        x = x.view(x.shape[0], -1)
#         print(x.shape)
        x = self.fc_layer(x)
#         print(x.shape)

        return x

In [8]:
adversarial_loss = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

In [9]:
generator

Generator(
  (rel): LeakyReLU(negative_slope=0.2, inplace=True)
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=2048, bias=True)
  )
  (deconv1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv3): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv4): ConvTranspose2d(16, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)

In [10]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [11]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (convs): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc_layer

In [12]:
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [13]:
dataset = datasets.ImageFolder(IMAGE_PATH, transform)
data_loader = torch.utils.data.DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)

In [14]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [15]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [19]:
print('training start!')
start_time = time.time()
for epoch in range(5,15):
    for i, (imgs, _) in enumerate(data_loader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        #  Train Generator
        
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        
        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if i % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, epochs, i, len(data_loader), d_loss.item(), g_loss.item())
            )
        batches_done = epoch * len(data_loader) + i
    save_image(gen_imgs.data[:16], "sample_images/%d.png" % epoch, nrow=4, normalize=True)
end_time = time.time()
total_time = end_time - start_time
total_time

training start!
[Epoch 5/5] [Batch 0/3165] [D loss: 0.608303] [G loss: 0.962178]
[Epoch 5/5] [Batch 100/3165] [D loss: 0.565683] [G loss: 0.769831]
[Epoch 5/5] [Batch 200/3165] [D loss: 0.568141] [G loss: 0.720198]
[Epoch 5/5] [Batch 300/3165] [D loss: 0.583818] [G loss: 1.069882]
[Epoch 5/5] [Batch 400/3165] [D loss: 0.540312] [G loss: 0.816347]
[Epoch 5/5] [Batch 500/3165] [D loss: 0.379157] [G loss: 1.450890]
[Epoch 5/5] [Batch 600/3165] [D loss: 0.457385] [G loss: 0.756448]
[Epoch 5/5] [Batch 700/3165] [D loss: 0.641378] [G loss: 0.999967]
[Epoch 5/5] [Batch 800/3165] [D loss: 0.717636] [G loss: 1.011828]
[Epoch 5/5] [Batch 900/3165] [D loss: 0.570377] [G loss: 0.907937]
[Epoch 5/5] [Batch 1000/3165] [D loss: 0.664378] [G loss: 0.572132]
[Epoch 5/5] [Batch 1100/3165] [D loss: 0.675641] [G loss: 0.846205]
[Epoch 5/5] [Batch 1200/3165] [D loss: 0.543508] [G loss: 1.029174]
[Epoch 5/5] [Batch 1300/3165] [D loss: 0.772108] [G loss: 0.781758]
[Epoch 5/5] [Batch 1400/3165] [D loss: 0.646

[Epoch 8/5] [Batch 2500/3165] [D loss: 0.722125] [G loss: 0.899239]
[Epoch 8/5] [Batch 2600/3165] [D loss: 0.586959] [G loss: 1.169443]
[Epoch 8/5] [Batch 2700/3165] [D loss: 0.676356] [G loss: 0.880139]
[Epoch 8/5] [Batch 2800/3165] [D loss: 0.569396] [G loss: 0.675996]
[Epoch 8/5] [Batch 2900/3165] [D loss: 0.482604] [G loss: 0.720669]
[Epoch 8/5] [Batch 3000/3165] [D loss: 0.593776] [G loss: 0.885963]
[Epoch 8/5] [Batch 3100/3165] [D loss: 0.516833] [G loss: 1.009062]
[Epoch 9/5] [Batch 0/3165] [D loss: 0.584358] [G loss: 1.004774]
[Epoch 9/5] [Batch 100/3165] [D loss: 0.806777] [G loss: 0.696592]
[Epoch 9/5] [Batch 200/3165] [D loss: 0.534036] [G loss: 1.008659]
[Epoch 9/5] [Batch 300/3165] [D loss: 0.540458] [G loss: 0.996163]
[Epoch 9/5] [Batch 400/3165] [D loss: 0.653046] [G loss: 0.796747]
[Epoch 9/5] [Batch 500/3165] [D loss: 0.768449] [G loss: 1.059432]
[Epoch 9/5] [Batch 600/3165] [D loss: 0.647552] [G loss: 1.207552]
[Epoch 9/5] [Batch 700/3165] [D loss: 0.643697] [G loss: 

[Epoch 12/5] [Batch 1700/3165] [D loss: 0.746452] [G loss: 0.658376]
[Epoch 12/5] [Batch 1800/3165] [D loss: 0.789595] [G loss: 1.179258]
[Epoch 12/5] [Batch 1900/3165] [D loss: 0.545387] [G loss: 1.483832]
[Epoch 12/5] [Batch 2000/3165] [D loss: 0.464230] [G loss: 0.859230]
[Epoch 12/5] [Batch 2100/3165] [D loss: 0.582333] [G loss: 0.848294]
[Epoch 12/5] [Batch 2200/3165] [D loss: 0.603481] [G loss: 0.948896]
[Epoch 12/5] [Batch 2300/3165] [D loss: 0.481259] [G loss: 1.148590]
[Epoch 12/5] [Batch 2400/3165] [D loss: 0.790206] [G loss: 0.805694]
[Epoch 12/5] [Batch 2500/3165] [D loss: 0.605837] [G loss: 0.984990]
[Epoch 12/5] [Batch 2600/3165] [D loss: 0.564756] [G loss: 0.741347]
[Epoch 12/5] [Batch 2700/3165] [D loss: 0.476353] [G loss: 0.745212]
[Epoch 12/5] [Batch 2800/3165] [D loss: 0.559584] [G loss: 1.565217]
[Epoch 12/5] [Batch 2900/3165] [D loss: 0.480121] [G loss: 0.779332]
[Epoch 12/5] [Batch 3000/3165] [D loss: 0.862112] [G loss: 0.656593]
[Epoch 12/5] [Batch 3100/3165] [D 

1076.4832208156586

In [17]:
epochs

5

In [21]:
images = []
for epoch in range(epochs):
    img_name = 'sample_images/' + str(epoch) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('sample_images/animation.gif', images, fps=2)